In [ ]:
import pulp

# 線形計画問題（Linear Programming, LP）を定式化するために PuLP を利用する
# LpProblem(name, sense) で最適化問題を生成する
# ここでは 'SLE' という名前で「最大化問題（LpMaximize）」として宣言している
problem = pulp.LpProblem('SLE', pulp.LpMaximize)

# 連続変数 x, y を定義する
# 線形計画法では、意思決定変数（decision variables）をこのように数理モデル上の変数として定義する
# cat='Continuous' は連続変数（実数値を取る）であることを意味する
x = pulp.LpVariable('x', cat='Continuous')
y = pulp.LpVariable('y', cat='Continuous')

# 以下の2行は、実質的には「制約条件」を追加している
# PuLP では `problem += 式 == 右辺` の形は「線形制約：式 = 右辺」を意味する
# 1つ目の制約: 120x + 150y = 1440
#   - 一般的な線形計画モデルでは、これは「資源制約」や「コスト・時間制約」などとして解釈される
#   - 数理的には、2変数の一次方程式であり、平面上の直線を表す
problem += 120 * x + 150 * y == 1440

# 2つ目の制約: x + y = 10
#   - これも2変数の一次方程式であり、別の直線を表す
#   - 2本の直線の交点が、この制約系を満たす (x, y) の組となる
problem += x + y == 10

# 注意（理論的な観点）:
# 本来、線形計画問題は
#   max / min     c^T z
#   subject to    A z (<=, =, >=) b
#   and           z in R^n (または z >= 0 など)
# のように「目的関数 c^T z」と「制約 Az = b」を同時に持つ。
# しかし、このコードでは "最大化問題" と宣言しているものの、
# 実質的には「目的関数」を別途 problem += ... で追加していないため、
# このモデルは「目的関数なしの実行可能性問題（A z = b の解を1つ見つける問題）」になっている。
# => 実質的には「連立一次方程式を解いている」のに近い状態になっている。

# ソルバーを呼び出し、線形計画問題（ここでは実質的には連立一次方程式）を解く
status = problem.solve()

# 解の状態を出力
# LpStatus[status] には、Optimal / Infeasible / Unbounded などの文字列が入る
print('Status:', pulp.LpStatus[status])

# 求まった変数 x, y の値を出力
# 連立一次方程式の観点では、2本の直線の交点座標 (x, y) を求めたことになる
print('x=', x.value(), 'y=', y.value())

In [4]:
# PythonライブラリPuLPの取り込み
import pulp

In [5]:
# 数理モデルの定義
problem = pulp.LpProblem("SLE", pulp.LpMaximize)
problem

SLE:
MAXIMIZE
None
VARIABLES

In [6]:
# 変数の定義
x = pulp.LpVariable("x", cat="Continuous")
y = pulp.LpVariable("y", cat="Continuous")

In [7]:
# 制約式の定義
problem += 120 * x + 150 * y == 1440
problem += x + y == 10
problem

SLE:
MAXIMIZE
None
SUBJECT TO
_C1: 120 x + 150 y = 1440

_C2: x + y = 10

VARIABLES
x free Continuous
y free Continuous

In [ ]:
# 求解
status = problem.solve()
print("Status:", pulp.LpStatus[status])

In [9]:
# 最適化結果の表示
print("x=", x.value(), "y=", y.value())

x= 2.0 y= 8.0


In [ ]:
import pulp

# 線形計画問題（Linear Programming, LP）を定義する
# 第1引数: 問題名 'LP'
# 第2引数: 目的が最大化か最小化か（ここでは LpMaximize = 最大化問題）
# 数理最適化でいうと「max c^T x, s.t. Ax <= b, x >= 0」という形式をコード化しているイメージ
problem = pulp.LpProblem("LP", pulp.LpMaximize)

# 意思決定変数（decision variables）x, y を連続変数（実数値）として定義
# デフォルトでは下限が 0 になることが多いが、ここでは後で制約として x >= 0, y >= 0 を明示する
# 理論的には「x, y は生産量などの非負連続量」と解釈できる
x = pulp.LpVariable("x", cat="Continuous")
y = pulp.LpVariable("y", cat="Continuous")

# --- 制約条件（constraints）の定義 ---
# 1つ目の制約: x + 3y <= 30
# 平面上では直線 x + 3y = 30 の「下側」の半平面を表す
# 線形計画の観点では、資源や時間の上限など「線形資源制約」として解釈できる
problem += 1 * x + 3 * y <= 30

# 2つ目の制約: 2x + y <= 40
# これも直線 2x + y = 40 の「下側」の半平面を表す線形制約
# 1つ目の制約と合わせて、実行可能領域（feasible region）の形を決める
problem += 2 * x + 1 * y <= 40

# 3つ目・4つ目の制約: x >= 0, y >= 0
# 変数を第1象限（非負領域）に制限する非負制約（non-negativity constraints）
# LP 理論では「標準形」で x >= 0 を課すのが一般的で、実行可能領域を凸多角形にする要素の一つ
problem += x >= 0
problem += y >= 0

# --- 目的関数（objective function）の定義 ---
# 目的関数: maximize x + 2y
# ベクトル表記では c = (1, 2)^T, 変数ベクトルを z = (x, y)^T とすると
#   max c^T z = max (1 * x + 2 * y)
# となる。幾何学的には「等高線が平行な直線」であり、その直線を
# 実行可能領域（凸多角形）の外側へ押し出していったときに最後に触れる頂点が最適解になる。
problem.objective = x + 2 * y

# ソルバーを呼び出して線形計画問題を解く
# 内部では単体法（Simplex 法）などのアルゴリズムが用いられ、
# 実行可能領域の「頂点（extreme points）」をたどりながら目的関数値を最大化する。
status = problem.solve()

# 解の状態（Optimal / Infeasible / Unbounded など）を出力
# Optimal なら「実行可能領域の中で目的関数を最大化する解が見つかった」ことを意味する
print("Status:", pulp.LpStatus[status])

# 求められた最適解 x*, y* と、そのときの目的関数値を出力
# LP の基本定理により、最適解は実行可能領域（線形制約の共通部分）を構成する
# 凸多角形の頂点のいずれかに存在するため、この出力はその頂点座標を与えていると解釈できる。
print("x=", x.value(), "y=", y.value(), "obj=", problem.objective.value())

In [12]:
# PythonライブラリPuLPの取り込み
import pulp

In [13]:
# 数理最適化モデルの定義
problem = pulp.LpProblem("LP", pulp.LpMaximize)
problem

LP:
MAXIMIZE
None
VARIABLES

In [14]:
# 変数の定義
x = pulp.LpVariable("x", cat="Continuous")
y = pulp.LpVariable("y", cat="Continuous")

In [15]:
# 制約式の定義
problem += 1 * x + 3 * y <= 30
problem += 2 * x + 1 * y <= 40
problem += x >= 0
problem += y >= 0

In [16]:
# 目的関数の定義
problem.objective = x + 2 * y
problem

LP:
MAXIMIZE
1*x + 2*y + 0
SUBJECT TO
_C1: x + 3 y <= 30

_C2: 2 x + y <= 40

_C3: x >= 0

_C4: y >= 0

VARIABLES
x free Continuous
y free Continuous

In [ ]:
# 求解
status = problem.solve()
print("Status:", pulp.LpStatus[status])

In [18]:
# 最適化結果の表示
print("x=", x.value(), "y=", y.value(), "obj=", problem.objective.value())

x= 18.0 y= 4.0 obj= 26.0
